### load_data 函数

In [1]:
import os
import json
import torch
from jupyter_server.transutils import base_dir
from torch_geometric.data import Data
import pandas as pd
import numpy as np

# 加载数据
def load_data(base_dir, epoch, step):
    # 用户位置
    user_pos_file = os.path.join(base_dir, f"UserEquipmentPosition/user_positions_{epoch}_{step}.json")
    with open(user_pos_file, 'r') as f:
        user_positions = json.load(f)

    # 基站位置
    bs_pos_file = os.path.join(base_dir, f"BaseStationPosition/stations_info_{epoch}_{step}.json")
    with open(bs_pos_file, 'r') as f:
        bs_positions = json.load(f)

    # 数据速率
    data_rate_file = os.path.join(base_dir, f"DataRate/data_rates_{epoch}_{step}.json")
    with open(data_rate_file, 'r') as f:
        data_rates = json.load(f)

    return user_positions, bs_positions, data_rates

### 测试 LoadData 函数

In [2]:
# 设置基础路径
base_dir = "/Users/yangpeilin/NUS CE/project/mobile-env-gan/collectData"

# 测试加载数据
epoch = 0
step = 0
user_positions, bs_positions, data_rates = load_data(base_dir, epoch, step)

# 打印加载的数据
print("User Positions:")
print(user_positions)
print("\nBase Station Positions:")
print(bs_positions)
print("\nData Rates:")
print(data_rates)

User Positions:
[{'ue_id': 0, 'x': 81.0, 'y': 109.0}, {'ue_id': 1, 'x': 142.0, 'y': 187.0}, {'ue_id': 2, 'x': 161.0, 'y': 86.0}, {'ue_id': 3, 'x': 156.0, 'y': 91.0}, {'ue_id': 4, 'x': 70.0, 'y': 21.0}, {'ue_id': 5, 'x': 10.0, 'y': 48.0}, {'ue_id': 6, 'x': 177.0, 'y': 108.0}]

Base Station Positions:
[{'bs_id': 0, 'x': 181.0, 'y': 153.0}, {'bs_id': 1, 'x': 75.0, 'y': 68.0}, {'bs_id': 2, 'x': 128.0, 'y': 32.0}, {'bs_id': 3, 'x': 74.0, 'y': 108.0}, {'bs_id': 4, 'x': 176.0, 'y': 115.0}]

Data Rates:
[{'ue_id': 1, 'bs_id': 0, 'data_rate': 0.84}, {'ue_id': 5, 'bs_id': 1, 'data_rate': 0.17}, {'ue_id': 4, 'bs_id': 1, 'data_rate': 0.57}, {'ue_id': 0, 'bs_id': 3, 'data_rate': 700.2}, {'ue_id': 2, 'bs_id': 4, 'data_rate': 1.33}, {'ue_id': 6, 'bs_id': 4, 'data_rate': 233.4}, {'ue_id': 3, 'bs_id': 4, 'data_rate': 1.55}]


### build_graph 函数

In [3]:
# 构建图数据
def build_graph(user_positions, bs_positions, data_rates):
    user_ids = [user['ue_id'] for user in user_positions]
    bs_ids = [bs['bs_id'] for bs in bs_positions]

    # 节点特征：用户和基站的位置 (x, y)
    user_features = torch.tensor([[user['x'], user['y']] for user in user_positions], dtype=torch.float)
    bs_features = torch.tensor([[bs['x'], bs['y']] for bs in bs_positions], dtype=torch.float)
    node_features = torch.cat([user_features, bs_features], dim=0)

    # 边的连接
    edges = []
    edge_weights = []
    edge_distances = []  # 用于存储距离特征
    
    # 建立所有可能的用户和基站之间的边
    for user in user_positions:
        ue_idx = user_ids.index(user['ue_id'])
        user_pos = torch.tensor([user['x'], user['y']], dtype=torch.float)
        for bs in bs_positions:
            bs_idx = len(user_positions) + bs_ids.index(bs['bs_id'])
            bs_pos = torch.tensor([bs['x'], bs['y']], dtype=torch.float)
            
            # 添加无向边
            edges.append([ue_idx, bs_idx])
            edges.append([bs_idx, ue_idx])
            
            # 初始化边的权重为 0
            edge_weights.append(0.0)
            edge_weights.append(0.0)
            
            # 计算用户与基站之间的欧几里得距离
            distance = torch.norm(user_pos - bs_pos).item()
            edge_distances.append(distance)
            edge_distances.append(distance)
    
    # 将真实数据速率 (data rate) 赋值到对应的边
    for connection in data_rates:
        ue_idx = user_ids.index(connection['ue_id'])
        bs_idx = len(user_positions) + bs_ids.index(connection['bs_id'])
        edge_idx = edges.index([ue_idx, bs_idx])  # 找到边的索引
        edge_weights[edge_idx] = connection['data_rate']
        edge_weights[edge_idx + 1] = connection['data_rate']  # 双向边

    # 构建 PyG 图结构
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(list(zip(edge_weights, edge_distances)), dtype=torch.float)  # 将权重和距离拼接

    # 返回 PyG Data 对象
    return Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr)


### 测试 build_graph 函数

In [4]:
# 测试构建图数据
graph_data = build_graph(user_positions, bs_positions, data_rates)

def print_graph_info(data):
    print("--- Graph Information ---")
    print(f"Number of nodes: {data.x.size(0)}")  # 节点数
    print(f"Number of edges: {data.edge_index.size(1)}")  # 边数
    print(f"data.x.shape: {data.x.shape}")  # 节点特征的 shape
    print(f"data.edge_index.shape: {data.edge_index.shape}")  # Edge Index 的 shape
    print(f"data.edge_attr.shape: {data.edge_attr.shape}")  # 边特征的 shape
    print("\nNode features (data.x):")
    print(data.x)  # 打印节点特征
    print("\nEdge Index (data.edge_index):")
    print(data.edge_index)  # 打印边连接
    print("\nEdge attributes (data.edge_attr):")
    print(data.edge_attr)  # 打印边特征矩阵

    print("\n--- Edge Connections ---")
    for i in range(data.edge_index.size(1)):  # 遍历每条边
        src = data.edge_index[0, i].item()
        dst = data.edge_index[1, i].item()
        edge_attr = data.edge_attr[i].tolist()  # 转换为 Python 列表
        print(f"Edge from Node {src} to Node {dst} with attributes {edge_attr}")
print_graph_info(graph_data)


--- Graph Information ---
Number of nodes: 12
Number of edges: 70
data.x.shape: torch.Size([12, 2])
data.edge_index.shape: torch.Size([2, 70])
data.edge_attr.shape: torch.Size([70, 2])

Node features (data.x):
tensor([[ 81., 109.],
        [142., 187.],
        [161.,  86.],
        [156.,  91.],
        [ 70.,  21.],
        [ 10.,  48.],
        [177., 108.],
        [181., 153.],
        [ 75.,  68.],
        [128.,  32.],
        [ 74., 108.],
        [176., 115.]])

Edge Index (data.edge_index):
tensor([[ 0,  7,  0,  8,  0,  9,  0, 10,  0, 11,  1,  7,  1,  8,  1,  9,  1, 10,
          1, 11,  2,  7,  2,  8,  2,  9,  2, 10,  2, 11,  3,  7,  3,  8,  3,  9,
          3, 10,  3, 11,  4,  7,  4,  8,  4,  9,  4, 10,  4, 11,  5,  7,  5,  8,
          5,  9,  5, 10,  5, 11,  6,  7,  6,  8,  6,  9,  6, 10,  6, 11],
        [ 7,  0,  8,  0,  9,  0, 10,  0, 11,  0,  7,  1,  8,  1,  9,  1, 10,  1,
         11,  1,  7,  2,  8,  2,  9,  2, 10,  2, 11,  2,  7,  3,  8,  3,  9,  3,
         10,  3

### 定义 GNN 模型

In [5]:
import torch
from torch.nn import Linear, Dropout, BatchNorm1d
from torch_geometric.nn import GATConv

class GNNModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, edge_dim, heads=4, dropout_rate=0.3):
        super(GNNModel, self).__init__()

        # 定义每一层 GATConv
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads, concat=True, edge_dim=edge_dim)
        self.conv2 = GATConv(hidden_channels * heads, hidden_channels, heads=heads, concat=True, edge_dim=edge_dim)
        self.conv3 = GATConv(hidden_channels * heads, hidden_channels, heads=heads, concat=True, edge_dim=edge_dim)
        self.conv4 = GATConv(hidden_channels * heads, hidden_channels, heads=heads, concat=True, edge_dim=edge_dim)
        self.conv5 = GATConv(hidden_channels * heads, out_channels, heads=1, concat=False, edge_dim=edge_dim)

        # 定义边特征的线性变换
        self.edge_mlp = Linear(2 * out_channels + edge_dim, 1)  # 注意拼接的维度

        # Dropout 和 BatchNorm
        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)
        self.dropout3 = Dropout(dropout_rate)
        self.dropout4 = Dropout(dropout_rate)

        self.batch_norm1 = BatchNorm1d(hidden_channels * heads)
        self.batch_norm2 = BatchNorm1d(hidden_channels * heads)
        self.batch_norm3 = BatchNorm1d(hidden_channels * heads)
        self.batch_norm4 = BatchNorm1d(hidden_channels * heads)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        # 第一层
        x = self.conv1(x, edge_index, edge_attr).relu()
        x = self.batch_norm1(x)
        x = self.dropout1(x)

        # 第二层
        x = self.conv2(x, edge_index, edge_attr).relu()
        x = self.batch_norm2(x)
        x = self.dropout2(x)

        # 第三层
        x = self.conv3(x, edge_index, edge_attr).relu()
        x = self.batch_norm3(x)
        x = self.dropout3(x)

        # 第四层
        x = self.conv4(x, edge_index, edge_attr).relu()
        x = self.batch_norm4(x)
        x = self.dropout4(x)

        # 第五层
        x = self.conv5(x, edge_index, edge_attr).relu()

        # 提取边的起点和终点特征，拼接边特征
        src, dst = edge_index
        edge_features = torch.cat([x[src], x[dst], edge_attr], dim=1)  # 拼接节点与边特征

        # 通过线性层预测边权重
        edge_weights = self.edge_mlp(edge_features)
        edge_weights = torch.relu(edge_weights)  # 修正负值预测问题
        return edge_weights.squeeze()

    def predict_edge_weights(self, node_features, edge_index, edge_attr):
        # 节点特征通过多层 GAT 提取高阶特征
        x = self.conv1(node_features, edge_index, edge_attr).relu()
        x = self.batch_norm1(x)
        x = self.dropout1(x)

        x = self.conv2(x, edge_index, edge_attr).relu()
        x = self.batch_norm2(x)
        x = self.dropout2(x)

        x = self.conv3(x, edge_index, edge_attr).relu()
        x = self.batch_norm3(x)
        x = self.dropout3(x)

        x = self.conv4(x, edge_index, edge_attr).relu()
        x = self.batch_norm4(x)
        x = self.dropout4(x)

        x = self.conv5(x, edge_index, edge_attr).relu()

        # 提取边的起点和终点特征，拼接边特征
        src, dst = edge_index
        edge_features = torch.cat([x[src], x[dst], edge_attr], dim=1)

        # 通过线性层预测边权重，并使用 ReLU 激活避免负数
        edge_weights = self.edge_mlp(edge_features)
        edge_weights = torch.relu(edge_weights)  # 修正负值预测问题

        return edge_weights.squeeze()

### 定义归一化函数

In [6]:
def normalize_data(data, scale_factor=None, target_min=None, target_max=None):
    
    # 针对数据速率特征 (data.edge_attr[:, 0])
    non_zero_mask = data.edge_attr[:, 0] > 0  # 非零边的掩码
    data_rate = data.edge_attr[non_zero_mask, 0]  # 仅提取非零数据速率

    if data_rate.numel() > 0:  # 确保非零边存在
        # 对数缩放
        log_scaled = torch.log1p(data_rate)  # log(1 + x)
        
        # 扩大
        if scale_factor is not None:
            data_rate = log_scaled * scale_factor
        else:
            data_rate = log_scaled

        if target_min is not None and target_max is not None:
            # 映射到 [target_min, target_max]
            log_min, log_max = data_rate.min(), data_rate.max()
            scale = (target_max - target_min) / (log_max - log_min + 1e-8)
            mapped = target_min + (data_rate - log_min) * scale
            data.edge_attr[non_zero_mask, 0] = mapped
        else:
            # 不做映射，仅放大
            data.edge_attr[non_zero_mask, 0] = data_rate

    return data


### 定义 MSE 损失函数

In [7]:
# 加权 MSE 损失函数（增加正则化）
def weighted_mse_loss(pred, target, weight, model, l2_lambda=1e-4):
    """
    加权 MSE 损失函数：
    - pred: 模型预测的标量边权重 (num_edges,)
    - target: 目标边权重 (num_edges,)，只选择 data_rate 列
    - weight: 动态计算的权重 (num_edges,)
    - model: 当前模型，用于计算 L2 正则化
    - l2_lambda: L2 正则化系数
    """
    # 提取目标的 data_rate 列
    target = target[:, 0]  # 只选择 data_rate 列

    # 加权 MSE 损失
    mse_loss = (weight * (pred - target) ** 2).mean()

    # L2 正则化项
    l2_reg = sum(torch.sum(param ** 2) for param in model.parameters())
    return mse_loss + l2_lambda * l2_reg


### 加载 DataLoader

In [8]:
from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader  # 确保使用的是 loader.DataLoader

# 加载数据
train_data = []
edge_stats = {"data_rate_non_zero": 0, "data_rate_total": 0}

for epoch in range(1000):  # 假设有1000个epoch数据
    for step in range(20):  # 假设每个epoch有20步
        try:
            user_positions, bs_positions, data_rates = load_data(base_dir, epoch, step)
            data = build_graph(user_positions, bs_positions, data_rates)
            
            # 统计非零数据速率
            edge_stats["data_rate_non_zero"] += (data.edge_attr[:, 0] > 0).sum().item()
            edge_stats["data_rate_total"] += data.edge_attr.size(0)
            
            # data = normalize_data(data)  # 数据归一化
            train_data.append(data)
        except FileNotFoundError:
            continue  # 跳过缺失文件
            
# 打印数据统计信息
non_zero_ratio = edge_stats['data_rate_non_zero'] / edge_stats['data_rate_total'] * 100
print(f"非零数据速率边占比: {non_zero_ratio:.2f}%")

# 划分训练集和验证集
train_data, valid_data = train_test_split(train_data, test_size=0.2, random_state=42)

# 再次统计训练集和验证集的非零数据速率
def calculate_edge_stats(dataset):
    stats = {"data_rate_non_zero": 0, "data_rate_total": 0}
    for data in dataset:
        stats["data_rate_non_zero"] += (data.edge_attr[:, 0] > 0).sum().item()
        stats["data_rate_total"] += data.edge_attr.size(0)
    return stats

train_stats = calculate_edge_stats(train_data)
valid_stats = calculate_edge_stats(valid_data)

train_non_zero_ratio = train_stats['data_rate_non_zero'] / train_stats['data_rate_total'] * 100
valid_non_zero_ratio = valid_stats['data_rate_non_zero'] / valid_stats['data_rate_total'] * 100

print(f"训练集非零数据速率边占比: {train_non_zero_ratio:.2f}%")
print(f"验证集非零数据速率边占比: {valid_non_zero_ratio:.2f}%")

# 数据加载器
batch_size = 64  # 调整 batch_size
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)

# 确定边特征维度
edge_dim = train_data[0].edge_attr.size(1)
assert all(data.edge_attr.size(1) == edge_dim for data in train_data + valid_data), "边特征维度不一致！"

非零数据速率边占比: 12.58%
训练集非零数据速率边占比: 12.57%
验证集非零数据速率边占比: 12.59%


### 查看 DataLoader 数据

In [9]:
def print_dataLoader_data(data):
    print("--- Graph Information ---")
    print(f"Number of nodes: {data.x.size(0)}")  # 节点数
    print(f"Number of edges: {data.edge_index.size(1)}")  # 边数
    print(f"data.x.shape: {data.x.shape}")  # 节点特征的 shape
    print(f"data.edge_index.shape: {data.edge_index.shape}")  # Edge Index 的 shape
    print(f"data.edge_attr.shape: {data.edge_attr.shape}")  # 边特征的 shape
    print("\nNode features (data.x):")
    print(data.x)  # 打印节点特征
    print("\nEdge Index (data.edge_index):")
    print(data.edge_index)  # 打印边连接
    print("\nEdge attributes (data.edge_attr):")
    print(data.edge_attr)  # 打印边特征矩阵

    print("\n--- Edge Connections ---")
    for i in range(data.edge_index.size(1)):  # 遍历每条边
        src = data.edge_index[0, i].item()
        dst = data.edge_index[1, i].item()
        edge_attr = data.edge_attr[i].tolist()  # 转换为 Python 列表
        print(f"Edge from Node {src} to Node {dst} with attributes {edge_attr}")

print_dataLoader_data(train_data[4])


--- Graph Information ---
Number of nodes: 17
Number of edges: 140
data.x.shape: torch.Size([17, 2])
data.edge_index.shape: torch.Size([2, 140])
data.edge_attr.shape: torch.Size([140, 2])

Node features (data.x):
tensor([[ 65.,  33.],
        [ 54.,  55.],
        [ 33., 150.],
        [ 97.,  25.],
        [ 36., 124.],
        [ 54., 144.],
        [ 43., 129.],
        [194., 153.],
        [150., 104.],
        [ 27.,  70.],
        [ 28., 186.],
        [ 26., 127.],
        [ 69., 172.],
        [ 23., 140.],
        [189.,  68.],
        [ 63., 183.],
        [ 31.,  48.]])

Edge Index (data.edge_index):
tensor([[ 0,  7,  0,  8,  0,  9,  0, 10,  0, 11,  0, 12,  0, 13,  0, 14,  0, 15,
          0, 16,  1,  7,  1,  8,  1,  9,  1, 10,  1, 11,  1, 12,  1, 13,  1, 14,
          1, 15,  1, 16,  2,  7,  2,  8,  2,  9,  2, 10,  2, 11,  2, 12,  2, 13,
          2, 14,  2, 15,  2, 16,  3,  7,  3,  8,  3,  9,  3, 10,  3, 11,  3, 12,
          3, 13,  3, 14,  3, 15,  3, 16,  4,  7,  4,  8, 

### 模型训练参数初始化

In [10]:
# 模型初始化
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNNModel(in_channels=2, hidden_channels=64, out_channels=16, edge_dim=edge_dim).to(device)

# 调整学习率和优化器
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-5)  # 增加 weight_decay 正则化
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.7)

### 准备文件

In [11]:
# 清空 results 文件夹
output_dir = "results_without_normolized"
if os.path.exists(output_dir):
    for file in os.listdir(output_dir):
        file_path = os.path.join(output_dir, file)
        if os.path.isfile(file_path):
            os.unlink(file_path)
else:
    os.makedirs(output_dir)

### 训练模型

In [12]:
import os
import torch
from torch_geometric.loader import DataLoader

# 模型训练和验证
num_epochs = 300

def write_validation_results(epoch, valid_loader, model, device, output_dir="results_without_normolized"):
    # 创建输出目录（如果不存在）
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    output_file = os.path.join(output_dir, f"result_epoch_{epoch}.txt")
    with open(output_file, "w") as f:
        model.eval()
        with torch.no_grad():
            valid_loss = 0
            valid_non_zero_loss_total = 0
            mae_total = 0
            mse_total = 0
            count = 0

            for batch_idx, batch in enumerate(valid_loader):
                batch = batch.to(device)
                pred_edge_weights = model(batch)

                # 验证损失计算（固定权重）
                fixed_weights = torch.where(
                    batch.edge_attr[:, 0] > 0,
                    torch.tensor(50.0, device=batch.edge_attr.device),
                    torch.tensor(0.1, device=batch.edge_attr.device)
                )
                loss = weighted_mse_loss(pred_edge_weights, batch.edge_attr, fixed_weights, model=model)
                valid_loss += loss.item()

                # 计算非零边损失（用于监控）
                mse_loss_fn = torch.nn.MSELoss()
                non_zero_loss = mse_loss_fn(
                    pred_edge_weights[batch.edge_attr[:, 0] > 0],
                    batch.edge_attr[batch.edge_attr[:, 0] > 0][:, 0]
                )
                valid_non_zero_loss_total += non_zero_loss.item()

                # 计算额外评估指标（MAE和MSE）
                non_zero_mae = torch.abs(
                    pred_edge_weights[batch.edge_attr[:, 0] > 0] -
                    batch.edge_attr[batch.edge_attr[:, 0] > 0][:, 0]
                ).mean().item()
                non_zero_mse = ((pred_edge_weights[batch.edge_attr[:, 0] > 0] -
                                 batch.edge_attr[batch.edge_attr[:, 0] > 0][:, 0]) ** 2).mean().item()

                mae_total += non_zero_mae
                mse_total += non_zero_mse
                count += 1

                # 写入对比数据
                f.write(f"--- Validation Batch {batch_idx + 1} ---\n")
                for i in range(min(100, batch.edge_attr.size(0))):  # 写入当前批次前100个样本
                    f.write(f"Edge {i}: True={batch.edge_attr[i, 0].item():.4f}, Pred={pred_edge_weights[i].item():.4f}\n")

            # 写入验证集损失和指标
            f.write(f"Validation at Epoch {epoch}:\n")
            f.write(f"  Validation Loss: {valid_loss / count:.4f}\n")
            f.write(f"  Validation Non-Zero Loss: {valid_non_zero_loss_total / count:.4f}\n")
            f.write(f"  MAE (Non-Zero Edges): {mae_total / count:.4f}, MSE (Non-Zero Edges): {mse_total / count:.4f}\n")

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    non_zero_loss_total = 0  # 非零权重边的总损失

    for batch in train_loader:
        batch = batch.to(device)

        # 前向传播
        pred_edge_weights = model(batch)

        # 动态计算权重（训练时使用）
        num_non_zero_edges = (batch.edge_attr[:, 0] > 0).sum()
        num_zero_edges = batch.edge_attr.size(0) - num_non_zero_edges

        non_zero_weight = 1.0 / (num_non_zero_edges + 1e-8)  # 防止除零
        zero_weight = 1.0 / (num_zero_edges + 1e-8)

        weights = torch.where(
            batch.edge_attr[:, 0] > 0,
            (30.0 * non_zero_weight).to(batch.edge_attr.device),
            (0.1 * zero_weight).to(batch.edge_attr.device)
        )

        # 调用自定义损失函数
        loss = weighted_mse_loss(pred_edge_weights, batch.edge_attr, weights, model=model)

        # 初始化标准 MSE 损失函数
        mse_loss_fn = torch.nn.MSELoss()

        # 计算非零边的损失（用于监控）
        non_zero_loss = mse_loss_fn(
            pred_edge_weights[batch.edge_attr[:, 0] > 0],
            batch.edge_attr[batch.edge_attr[:, 0] > 0][:, 0]  # 选择 data_rate 列
        )
        non_zero_loss_total += non_zero_loss.item()

        # 反向传播与优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # 学习率调度
    scheduler.step()

    # 打印训练损失
    print(f"Epoch {epoch + 1}/{num_epochs}, Total Loss: {total_loss:.4f}, Non-Zero Loss: {non_zero_loss_total:.4f}")

    # 每隔 10 个 epoch 进行验证，并保存结果
    if (epoch + 1) % 10 == 0:
        write_validation_results(epoch + 1, valid_loader, model, device)


Epoch 1/300, Total Loss: 206207837328086.5625, Non-Zero Loss: 46619006864139040.0000
Epoch 2/300, Total Loss: 198147818394003.9375, Non-Zero Loss: 44022261448120472.0000
Epoch 3/300, Total Loss: 185862857231094.7188, Non-Zero Loss: 41431657803357696.0000
Epoch 4/300, Total Loss: 174049370748453.5000, Non-Zero Loss: 38514079735571392.0000
Epoch 5/300, Total Loss: 159819059628674.6875, Non-Zero Loss: 35717522513599216.0000
Epoch 6/300, Total Loss: 147322757423387.2500, Non-Zero Loss: 33026015499968584.0000
Epoch 7/300, Total Loss: 136883494384537.1094, Non-Zero Loss: 30683509003902124.0000
Epoch 8/300, Total Loss: 129793891609549.2188, Non-Zero Loss: 28733575830824708.0000
Epoch 9/300, Total Loss: 119830913620449.5781, Non-Zero Loss: 26652642106516748.0000
Epoch 10/300, Total Loss: 110711476917035.4375, Non-Zero Loss: 24609232737312784.0000
Epoch 11/300, Total Loss: 102075429393697.9375, Non-Zero Loss: 22741535196437164.0000
Epoch 12/300, Total Loss: 94402101180837.1094, Non-Zero Loss: 2